In [49]:
import os
from tika import parser
import re
import pandas as pd
from bs4 import BeautifulSoup

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


path = os.getcwd()


In [50]:
path = r"C:\Users\luiz\Desktop\SERGIO"
path=path+"\\"

In [4]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
for pdf in folders:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
        #print("ALERTA",pdf)
    folhas = re.findall(r"Data pregão(.*?)XP",a.replace('\n',''))
    aux = '_'.join(folhas[0].split('/')[::-1])
    #print(aux)
    os.rename(path+pdf,path+aux+".pdf")

NameError: name 'os' is not defined

In [51]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(r"C:\Users\luiz\Desktop\flavio\nota_2\vendas\\")))
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
for pdf in folders:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    #########
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
    if 'WIN' in a:
        print("ALERTA WIN",pdf)
    if 'IND '  in a:
        print("ALERTA IND",pdf)
    ##########
    folhas = re.findall(r"NOTA DE CORRETAGEMNr. nota(.*?)Folha(.*?)Data pregão(.*?)XP",a.replace('\n',''))

    taxas = re.findall(r"Precatório\n([(0-9|.|,)]*)Total CBLC (D|C)\n([(0-9|.|,)]*)Valor líquido das operações (D|C)\n([(0-9|.|,)]*)Taxa de liquidação (D|C)\n([(0-9|.|,)]*)Taxa de Registro (D|C)\n([(0-9|.|,)]*)Total Bovespa / Soma (D|C)\n([(0-9|.|,)]*)Taxa de termo/opções (D|C)\n([(0-9|.|,)]*)Taxa A.N.A. (D|C)\n([(0-9|.|,)]*)Emolumentos (D|C)\n \n([(0-9|.|,)]*)Total corretagem / Despesas (D|C)\n \n([(0-9|.|,)]*)Taxa Operacional (D|C)\n([(0-9|.|,)]*)Execução\n([(0-9|.|,)]*)Taxa de Custódia\n([(0-9|.|,)]*)Impostos (D|C)\n([(0-9|.|,)]*)I.R.R.F. s/ operações, base R\$(.*?)\n([(0-9|.|,)]*)Outros (C|D)\n([(0-9|.|,)]*)Líquido para (.*?) (C|D)",a)
    #print(taxas)
    for i in range(len(taxas)):
        taxas[i] = list(taxas[i])
        if taxas[i][-6][-1] != "D":
            taxas[i][-7] = '0,00'
        assert taxas[i][20] == taxas[i][20] =='0,00'
        for j in range(15):
            if taxas[i][2*j+1][-1] == 'D':
                taxas[i][2*j] = '-'+taxas[i][2*j]
        if taxas[i][-1] == 'D':
                taxas[i][-3] = '-'+taxas[i][-3]
        taxas[i] = list(map(lambda x : int(x.replace('.','').replace(',','')) if not '/' in x else '-'.join(x.split('/')[::-1]),taxas[i][:-1:2] +[taxas[i][-2]]+[taxas[i][-6].split(' ')[0]]))
        try:
            lucro_daytrade,IRRF_daytrade = re.findall(r"IRRF Day Trade: Base R\$ (.*?) Projeção R\$ (.*?)\n",a)[0]
            taxas[i].append(-int(lucro_daytrade.replace('.','').replace(',','')))
            taxas[i].append(-int(IRRF_daytrade.replace('.','').replace(',','')))
        except:
            taxas[i].append(0)
            taxas[i].append(0)
        assert taxas[i][0] == sum(taxas[i][1:4])
        assert taxas[i][4] == sum(taxas[i][5:8])
        assert taxas[i][8] == sum(taxas[i][9:14])
        assert taxas[i][0] + taxas[i][4]+ taxas[i][8] + taxas[i][-1]== taxas[i][-5]
    #for k in taxas:
    #    print(k[-1],k[-5])
    #    print((k[1]*(1+0.00004105+0.0002795)+k[9]*(0.1068+0.039+1))/100 - k[-3]/100)

    trades = re.findall(r"Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/C(.*?)NOTA DE CORRETAGEM",a.replace('\n',''))
    trades2 = re.findall(r"CONTINUA...Corretagem / DespesasLíquido paraObservação: \(1\) As operações a termo não são computadas no líquido da fatura.Capitais e regiões metropolitanas: 4003-3710 Demais localidades: 0800-880-3710 SAC: 0800-772-0202 Ouvidoria: 0800-722-3710(.*?)Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/CNOTA DE CORRETAGEM",a.replace('\n',''))

    assert trades.count('') == len(trades2)
    for i,j in enumerate(folhas):
        if j[1] != '1':
            assert trades[i] == ''
            trades[i] = trades2[0]
            trades2 = trades2[1:]


        aux = filter(lambda x: x !='',trades[i].split('1-'))
        aux2 = map(lambda x : re.findall(r"^(.*?) (C|V) (.*?) (.*?) ([(0-9|.|,)]*) ([(0-9|.|,)]*) ([(0-9|.|,)]*) (D|C)$",x)[0],aux)
        aux3 = map(lambda x : [x[0],x[1],x[2],re.sub(' +', ' ', x[3]).strip(),x[4].replace('.','').replace(',','.'),x[5].replace('.','').replace(',','.'),x[6].replace('.','').replace(',','.'),x[7]],aux2)
        folhas[i] = [j[0],j[1],'-'.join(j[2].split('/')[::-1]),list(aux3)] 

    assert len(trades2)==0

    folhas = folhas[::-1]
    for i,k in enumerate(folhas):
        #Aqui to perdendo a info do marcado D do daytrade no nome do ativo
        k[3] = list(map(lambda x : (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int('-'+x[4]),x[5],int(x[6].replace('.',''))) if x[1] == 'V' else (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int(x[4]),x[5],int('-'+x[6].replace('.',''))) ,k[3]))
        if i != 0:
            if folhas[i-1][1] != '1':
                folhas[i][3]= folhas[i][3]+folhas[i-1][3]
    folhas = list(filter(lambda x: x[1] == '1',folhas[::-1]))
    folhas = list(map(lambda x: x[3],folhas))
    assert len(folhas)==len(taxas)

    historico.append([n for m in folhas for n in m])
    taxa.append(taxas)
historico,taxa = [n for m in historico for n in m],[z for x in taxa for z in x]


ALERTA IND 2020_04_08.pdf
ALERTA IND 2020_04_09.pdf
ALERTA IND 2020_04_16.pdf
ALERTA WIN 2020_04_27.pdf
ALERTA IND 2020_04_27.pdf
ALERTA WIN 2020_04_28.pdf
ALERTA IND 2020_04_28.pdf
ALERTA WIN 2020_04_29.pdf
ALERTA IND 2020_04_29.pdf
ALERTA WIN 2020_04_30.pdf
ALERTA IND 2020_04_30.pdf
ALERTA WIN 2020_06_03.pdf
ALERTA IND 2020_06_03.pdf
ALERTA WIN 2020_06_15.pdf
ALERTA WIN 2020_06_16.pdf
ALERTA WIN 2020_06_17.pdf
ALERTA WIN 2020_06_18.pdf


In [52]:

def translate_tickers(nome):
    tickers={
        'BTGP BANCO UNT N2':'BPAC11',
        'BTGP BANCO UNT N2 D':'BPAC11',
        'ENEVA ON NM':'ENEV3',
        'ENEVA ON NM D':'ENEV3',
        'BIOTOSCANA DR3':'GBIO33',
        'BIOTOSCANA DR3 D#2':'GBIO33',
        'BIOTOSCANA DR3 D':'GBIO33',
        'QUALICORP ON ER NM':'QUAL3',
        'QUALICORP ON NM':'QUAL3',
        'BRASIL ON NM #2D':'BBAS3',
        'BRASIL ON NM':'BBAS3',
        'BRASIL ON NM D':'BBAS3',
        'SINQIA ON NM':'SQIA3',
        'AES TIETE E UNT ED N2':'TIET11',
        'AES TIETE E UNT N2':'TIET11',
        'BRADESCO PN EJB N1':'BBDC4',
        'BRADESCO PN N1':'BBDC4',
        'BRADESCO ON EJB N1':'BBDC3',
        'BRADESCO ON N1':'BBDC3',
        'ITAUUNIBANCO PN ED N1':'ITUB4',
        'ITAUUNIBANCO PN N1':'ITUB4',
        'JBS ON ED NM':'JBSS3',
        'JBS ON NM':'JBSS3',
        'LOJAS RENNER ON EDB NM':'LREN3',
        'PETROBRAS PN N2 D':'PETR4',
        'LOJAS RENNER ON NM':'LREN3',
        'P.ACUCAR-CBD PN EJ N1':'PCAR4',
        'P.ACUCAR-CBD PN N1':'PCAR4',
        'PETROBRAS PN ED N2':'PETR4',
        'PETROBRAS PN N2':'PETR4',
        'BRASIL ON NM D':'BBAS3',
        'VALE ON NM':'VALE3',
        'AZUL PN N2':'AZUL4',
        'OI ON N1':'OIBR3',
        'OI ON N1 D':'OIBR3',
        'OI ON N1 D#2':'OIBR3',
        'B2W DIGITAL ON NM':'BTOW3',
        'ENGIE BRASIL ON NM':'EGIE3',
        'EQUATORIAL ON NM':'EQTL3',
        'LOCALIZA ON NM':'RENT3',
        'SUZANO PAPEL ON NM':'SUZB3',
        'CARREFOUR BR ON NM':'CRFB3',
        'TIM PART S/A ON NM':'TIMP3',
        'SUL AMERICA UNT N2':'SULA11',
        'MULTIPLAN ON N2':'MULT3',
        'TRAN PAULIST PN N1':'TRPL4',
        'RUMO S.A. ON NM':'RAIL3',
        'LOJAS RENNER ON ED NM D':'LREN3',
        'EZTEC ON NM':'EZTC3',
        'EZTEC ON EDB NM':'EZTC3',
        'B3 ON NM':'B3SA3',
        'ISHARES BOVA CI ATZ':'BOVA11',
        'LOCAWEB ON NM':'LWSA3',
        'RANDON PART PN N1':'RAPT4',
        'IGUATEMI ON NM' : 'IGTA3',
        'BR MALLS PAR ON NM':'BRML3',
        'MAGAZ LUIZA ON NM D':'MGLU3',
        'CVC BRASIL ON NM':'CVCB3',
        'TOTVS ON NM':'TOTS3',
        'KEPLER WEBER ON EJ':'KEPL3',
        'KEPLER WEBER ON':'KEPL3',
        'TUPY ON NM':'TUPY3',
        'PETRORIO ON NM':'PRIO3',
        'PETRORIO ON NM D':'PRIO3',
        'PETRORIO ON NM D#2':'PRIO3',
        'VIAVAREJO ON NM':'VVAR3',
        'CENTAURO ON NM':'CNTO3',
        'ELETROBRAS ON N1':'ELET3',
        'BRADESCO PN EJ N1':'BBDC4',
        'M.DIASBRANCO ON EJ NM':'MDIA3',
        'ABC BRASIL PN EJ N2':'ABCB4',
        'CAMIL ON NM':'CAML3',
        'GRUPO NATURA ON NM':'NTCO3',
        'CEA MODAS ON EJ NM':'CEAB3',
        'CEA MODAS ON NM':'CEAB3',
        'CIA HERING ON NM':'HGTX3',
        'IMC S/A ON NM':'MEAL3',
        'PPLA UNT':'PPLA11',
        'BANCO BMG PN N1':'BMGB4',
        'LOG-IN ON NM':'LOGN3',
        'DE COMPRA 01/20 VVARA125 ON 12,50 VVAR':'VVAR125',
        'OPC COMPRA 01/20 VVARA125E ON 12,50':'VVAR125',
        'PORTOBELLO ON NM':'PTBL3',
        'RECRUSUL PN':'RCSL4',
        'GAFISA ON NM':'GFSA3',
        'COGNA ON ON NM':'COGN3',
        'MILLS ON NM':'MILS3',
        'TECNISA ON NM':'TCSA3',
        'TRISUL ON NM' : 'TRIS3',
        'CEMIG PN N1' : 'CMIG4',
        'CEMIG PN EJ N1': 'CMIG4',
        'DOMMO ON':'DMMO3',
        'PETROBRAS BR ON NM':'BRDT3',
        'OI PN N1':'OIBR4',
        'ISHARES BOVA CI':'BOVA11',
        'ISHARES BOVA CI D':'BOVA11',
        'ELETROBRAS PNB N1':'ELET6',
        'FII XP CRED XPCI12 DM 100,51':'DSHFOSDHFOISDFJSDPOOFJSPOFJSDPF',
    }
    return tickers[nome]


In [53]:
aux = pd.DataFrame(historico,columns=['nota','pregao','nome','qtd','preco_medio','financeiro'])
aux['daytrade'] = aux.nome.apply(lambda x : True if x[-1] == 'D' or x[-3:-1] == 'D#' or x[-4:-1] == 'D#2' else False)
aux.nome = aux.nome.apply(translate_tickers)
aux['mes'] = aux['pregao'].apply(lambda x : '-'.join(x.split('-')[:2]))
aux = aux.sort_values(by='pregao',ascending=True)
aux.index = range(len(aux))
aux['turnover'] = abs(aux['financeiro'])



In [54]:
aux

,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
0,20630020,2020-04-06,ENEV3,200,32.97,-659400,True,2020-04,659400
1,20630020,2020-04-06,ENEV3,-300,33.33,999900,True,2020-04,999900
2,20630020,2020-04-06,ENEV3,-200,33.33,666600,True,2020-04,666600
3,20630020,2020-04-06,ENEV3,-3600,33.31,11991600,True,2020-04,11991600
4,20630020,2020-04-06,ENEV3,-100,33.31,333100,True,2020-04,333100
...,...,...,...,...,...,...,...,...,...
1076,24433627,2020-07-31,OIBR3,-400,1.76,70400,False,2020-07,70400
1077,24433627,2020-07-31,OIBR3,-300,1.76,52800,False,2020-07,52800
1078,24433627,2020-07-31,OIBR3,-2000,1.76,352000,False,2020-07,352000
1079,24433627,2020-07-31,OIBR3,-15500,1.76,2728000,False,2020-07,2728000


In [55]:
turnover = aux.groupby(by=['pregao','nota']).sum()[["turnover"]].sort_values(by='pregao',ascending=True)
turnover

,,turnover
pregao,nota,
2020-04-06,20630020,59671100
2020-04-07,20665692,15599300
2020-04-08,20702999,181761000
2020-04-09,20739679,33533400
2020-04-16,20888575,219338700
2020-04-24,21094769,42083500
2020-04-29,21232388,100620000
2020-04-30,21276284,40452000
2020-05-04,21318905,203174400


In [56]:
aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)


,pregao,nota
0,2020-04-06,20630020
1,2020-04-07,20665692
2,2020-04-08,20702999
3,2020-04-09,20739679
4,2020-04-16,20888575
5,2020-04-24,21094769
6,2020-04-29,21232388
7,2020-04-30,21276284
8,2020-05-04,21318905
9,2020-05-06,21398427


In [57]:
df_taxas = pd.DataFrame(taxa).sort_values(by=15,ascending=True)
df_taxas.columns = ['CBLC','valor_liquido_operacoes','taxa_liquidacao','taxa_registro','total_bovespa','taxa_termo_opcoes','taxa_ANA','emolumentos','total_custos','taxa_operacional','execucao','impostos','IRRF','outros','valor_liquido_para','data_liquidacao','montante_IRRF','lucro_daytrade','IRRF_daytrade']
df_taxas.index = range(len(df_taxas))
#df_taxas = df_taxas[[15,14]]
#df_taxas.columns = ['liquidacao','IRRF','valorIRRF','corretagem','valor_final','valor_liquid_operacoes','lucro_daytrade','IRRF_daytrade']
#df_taxas['valor_liquido_para'] = df_taxas['valor_liquido_para'] - df_taxas['IRRF_daytrade']
df_taxas['custos_operacionais'] = df_taxas['taxa_liquidacao']+df_taxas['emolumentos']+df_taxas['impostos']+df_taxas['outros']+df_taxas['taxa_operacional']
df_taxas = pd.concat([df_taxas,aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)],axis=1)
#df_taxas= df_taxas[['nota','pregao','data_liquidacao','taxa_operacional','custos_operacionais','valor_liquido_para']]
#df_taxas.set_index(15).T
df_taxas

,CBLC,valor_liquido_operacoes,taxa_liquidacao,taxa_registro,total_bovespa,taxa_termo_opcoes,taxa_ANA,emolumentos,total_custos,taxa_operacional,...,IRRF,outros,valor_liquido_para,data_liquidacao,montante_IRRF,lucro_daytrade,IRRF_daytrade,custos_operacionais,pregao,nota
0,313166,325100,-11934,0,-1793,0,0,-1793,-7791,-6800,...,0,-265,300537,2020-04-08,0,-304573,-3045,-21518,2020-04-06,20630020
1,-15603589,-15599300,-4289,0,-468,0,0,-468,-6496,-5670,...,0,-221,-15610553,2020-04-09,0,0,0,-11253,2020-04-07,20665692
2,-43872689,-43833000,-39689,0,-5463,0,0,-5463,-9829,-8579,...,0,-334,-43894220,2020-04-13,0,-623942,-6239,-54981,2020-04-08,20702999
3,33524179,33533400,-9221,0,-1008,0,0,-1008,-3840,-1890,...,-1676,-73,33519331,2020-04-14,33533400,0,0,-12393,2020-04-09,20739679
4,-219399018,-219338700,-60318,0,-6593,0,0,-6593,-8661,-7560,...,0,-294,-219414272,2020-04-20,0,0,0,-75572,2020-04-16,20888575
5,42071928,42083500,-11572,0,-1265,0,0,-1265,-6434,-3780,...,-2104,-147,42064229,2020-04-28,42083500,0,0,-17167,2020-04-24,21094769
6,100592330,100620000,-27670,0,-7043,0,0,-7043,-7195,-1890,...,-5031,-73,100578092,2020-05-04,100620000,0,0,-36877,2020-04-29,21232388
7,-40463124,-40452000,-11124,0,-2831,0,0,-2831,-6496,-5670,...,0,-221,-40472451,2020-05-05,0,0,0,-20451,2020-04-30,21276284
8,3133966,3174600,-40634,0,-6107,0,0,-6107,-2749,-2400,...,0,-93,3093856,2020-05-06,0,-3125460,-31254,-49490,2020-05-04,21318905
9,21497787,21503700,-5913,0,-718,0,0,-718,-4987,-3780,...,-657,-147,21492082,2020-05-08,13149000,0,0,-10961,2020-05-06,21398427


In [58]:
#verifica se os daytrades estão todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

qtd  financeiro  daytrade   turnover
pregao     nome                                         
2020-04-06 ENEV3      0      325100      42.0   59671100
2020-04-08 BPAC11     0      770000      14.0   76750000
           PETR4      0     -108000      14.0   60516000
2020-05-04 OIBR3      0     3174600      51.0  203174400
2020-05-26 BPAC11     0    -1475500     113.0  201567500
           MGLU3      0      458100      65.0  127840500
           PRIO3      0     -303600      44.0   60836400
2020-06-15 BOVA11 -7890    70215580      10.0  210609840
2020-06-16 BPAC11     0       -4700       4.0   12842700

In [59]:
#preciso na unha tirar os BOVA11 q na vdd n foram daytrade
confusao = aux[aux.daytrade][aux.nome == 'BOVA11']
confusao

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
860,22678282,2020-06-15,BOVA11,-1000,88.99,8899000,True,2020-06,8899000
862,22678282,2020-06-15,BOVA11,-10,88.98,88980,True,2020-06,88980
863,22678282,2020-06-15,BOVA11,-70,88.99,622930,True,2020-06,622930
864,22678282,2020-06-15,BOVA11,-1190,88.99,10589810,True,2020-06,10589810
865,22678282,2020-06-15,BOVA11,-400,88.98,3559200,True,2020-06,3559200
866,22678282,2020-06-15,BOVA11,-10,88.99,88990,True,2020-06,88990
867,22678282,2020-06-15,BOVA11,-13100,88.98,116563800,True,2020-06,116563800
868,22678282,2020-06-15,BOVA11,7680,88.97,-68328960,True,2020-06,68328960
869,22678282,2020-06-15,BOVA11,200,88.96,-1779200,True,2020-06,1779200
870,22678282,2020-06-15,BOVA11,10,88.97,-88970,True,2020-06,88970


In [60]:
qtd_comprada = confusao[confusao.qtd>0].qtd.sum()
qtd_vendida  = -confusao[confusao.qtd<0].qtd.sum()

#saldo final dos trades daquele dia, se ficou net comprado ou net vendido
saldo = qtd_comprada-qtd_vendida

#medio de compra
medio_compra = -0.01*confusao[confusao.qtd>0].financeiro.sum()/qtd_comprada
#medio_venda
medio_venda = 0.01*confusao[confusao.qtd<0].financeiro.sum()/qtd_vendida
medio_compra,medio_venda


(88.96974651457542, 88.98143852978454)

In [61]:
adicionar = confusao.iloc[0:2].copy()
adicionar.index = [0,1]
adicionar

,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
0,22678282,2020-06-15,BOVA11,-1000,88.99,8899000,True,2020-06,8899000
1,22678282,2020-06-15,BOVA11,-10,88.98,88980,True,2020-06,88980


In [62]:

if saldo > 0 and medio_compra < medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra < medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

elif saldo > 0 and medio_compra > medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra > medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

    
else:
    print('algum erro de caso inesperado')


#adicionar.index = [len(aux),len(aux)+1]
adicionar['turnover']  = adicionar['financeiro'].apply(abs)

In [63]:
aux.drop(confusao.index,inplace=True)
aux = pd.concat([aux,adicionar]).sort_values('pregao')

In [84]:
#agora todos os daytrades devem estar todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

qtd  financeiro  daytrade   turnover
pregao     nome                                        
2020-04-06 ENEV3     0      325100      42.0   59671100
2020-04-08 BPAC11    0      770000      14.0   76750000
           PETR4     0     -108000      14.0   60516000
2020-05-04 OIBR3     0     3174600      51.0  203174400
2020-05-26 BPAC11    0    -1475500     113.0  201567500
           MGLU3     0      458100      65.0  127840500
           PRIO3     0     -303600      44.0   60836400
2020-06-15 BOVA11    0        9225       1.0       9225
2020-06-16 BPAC11    0       -4700       4.0   12842700

In [85]:
#Quantidade de dinheiro ganhado no daytrade BVM (SEM INDICIE FUTURO) (ainda não estao incluidos os custos)
e['financeiro'].sum()/100

28452.25

In [ ]:
taxas_detalhadas = df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]
taxas_detalhadas

In [86]:
assert (df_taxas.valor_liquido_operacoes+df_taxas.custos_operacionais+df_taxas.IRRF+df_taxas.IRRF_daytrade == df_taxas.valor_liquido_para).all()
assert (aux.groupby(['nota']).sum().financeiro.values == taxas_detalhadas.valor_liquido_operacoes.values).all()

In [118]:
df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]

,nota,pregao,data_liquidacao,valor_liquido_operacoes,custos_operacionais,valor_liquido_para,IRRF,IRRF_daytrade
0,20630020,2020-04-06,2020-04-08,325100,-21518,300537,0,-3045
1,20665692,2020-04-07,2020-04-09,-15599300,-11253,-15610553,0,0
2,20702999,2020-04-08,2020-04-13,-43833000,-54981,-43894220,0,-6239
3,20739679,2020-04-09,2020-04-14,33533400,-12393,33519331,-1676,0
4,20888575,2020-04-16,2020-04-20,-219338700,-75572,-219414272,0,0
5,21094769,2020-04-24,2020-04-28,42083500,-17167,42064229,-2104,0
6,21232388,2020-04-29,2020-05-04,100620000,-36877,100578092,-5031,0
7,21276284,2020-04-30,2020-05-05,-40452000,-20451,-40472451,0,0
8,21318905,2020-05-04,2020-05-06,3174600,-49490,3093856,0,-31254
9,21398427,2020-05-06,2020-05-08,21503700,-10961,21492082,-657,0


In [161]:
df_taxas.IRRF.sum()/100,df_taxas.IRRF_daytrade.sum()/100,taxas_detalhadas[['IRRF','IRRF_daytrade']].sum().sum()/100

(-336.38, -405.38, -741.76)

In [91]:
teste = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
teste

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000


In [92]:
#valor em REAIS que foi cobrado a mais no 'taxa_liquidacoo'
print((-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795).sum()/100)
(-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795)/100

-819.2256577000003


0     -47.440724
1      -0.710044
2    -111.131995
3      -1.515853
4      -9.871667
5      -1.903383
6      -4.532900
7      -1.823340
8    -161.532448
9      -0.972842
10     -1.207865
11     -4.104159
12     -0.815400
13   -310.253098
14     -5.046397
15     -5.959123
16   -120.865687
17    -11.927617
18     -1.531857
19     -6.672323
20     -2.319460
21     -1.044000
22     -0.253214
23     -0.231305
24     -0.226800
25     -5.332160
dtype: float64

In [93]:
#valor em REAIS que foi cobrado a mais no 'emolumento'
print((-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105).sum()/100)
(-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105)/100

-153.64865563000004


0     -6.564987
1     -1.723513
2    -19.982891
3     -3.685461
4    -24.108536
5     -4.625277
6     29.125490
7     11.704454
8    -22.333091
9     -1.647269
10    -2.049643
11    -6.975031
12    -1.388260
13   -29.855326
14    -1.892814
15   -10.774274
16   -18.165000
17    -4.145732
18    -2.770210
19   -12.072463
20    -4.184774
21    -1.893600
22    -0.568136
23    -0.510489
24    -0.518920
25   -12.042904
dtype: float64

In [94]:
#valor em centavos que foi cobrado a mais no 'imposto'
-df_taxas.impostos+(0.1068)*df_taxas.reset_index()['taxa_operacional']

0    -0.2400
1    -0.5560
2    -0.2372
3    -0.8520
4    -0.4080
5    -0.7040
6    -0.8520
7    -0.5560
8    -0.3200
9    -0.7040
10   -0.7040
11   -0.6680
12   -0.8520
13   -0.2400
14   -0.7080
15   -0.6320
16   -0.5680
17   -0.5800
18   -0.8520
19   -0.4080
20   -0.8520
21   -0.8520
22   -0.8520
23   -0.7452
24   -0.7452
25   -0.7452
dtype: float64

In [95]:
#valor em centavos que foi cobrado a mais na 'outros'
-df_taxas.outros+(0.039)*df_taxas.reset_index()['taxa_operacional']

0    -0.200
1    -0.130
2    -0.581
3    -0.710
4    -0.840
5    -0.420
6    -0.710
7    -0.130
8    -0.600
9    -0.420
10   -0.420
11   -0.390
12   -0.710
13   -0.200
14   -0.590
15   -0.360
16   -0.640
17   -0.150
18   -0.710
19   -0.840
20   -0.710
21   -0.710
22   -0.710
23   -0.671
24   -0.671
25   -0.671
dtype: float64

In [96]:
#taxa_liquidacoo
-turnover.reset_index().turnover*0.0002795/100
#emolumentos
-turnover.reset_index().turnover*0.00004105/100
#impostos
(0.1068)*df_taxas.reset_index()['taxa_operacional']/100
#outros
(0.039)*df_taxas.reset_index()['taxa_operacional']/100

0     -2.65200
1     -2.21130
2     -3.34581
3     -0.73710
4     -2.94840
5     -1.47420
6     -0.73710
7     -2.21130
8     -0.93600
9     -1.47420
10    -1.47420
11    -6.63390
12    -0.73710
13    -6.55200
14   -21.37590
15   -11.79360
16    -3.41640
17    -4.62150
18    -0.73710
19    -2.94840
20    -0.73710
21    -0.73710
22    -0.73710
23    -0.73671
24    -0.73671
25    -0.73671
Name: taxa_operacional, dtype: float64

In [99]:
custo_distribuido = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
custo_distribuido['custo'] = 0
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,0
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,0
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,0
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,0
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,0
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,0
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,0
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,0
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,0
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,0


In [100]:
for nota in set(teste.nota):
    constante = custo_distribuido[custo_distribuido.nota == nota].turnover.sum()
    custo_total = taxas_detalhadas[taxas_detalhadas.nota == nota].custos_operacionais.sum()
    fator = custo_total/constante
    custo_distribuido['custo'] = custo_distribuido['custo']+ custo_distribuido.nota.apply(lambda x : fator if x==nota else 0)*custo_distribuido.turnover

In [101]:
#valor infimo
custo_distribuido.groupby('nota')[['custo']].sum().values - taxas_detalhadas.groupby('nota')[['custos_operacionais']].sum().values

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.45519152e-11],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 4.54747351e-13],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])

In [102]:
custo_distribuido = custo_distribuido.merge(taxas_detalhadas[['nota','data_liquidacao']],left_on='nota', right_on='nota')
custo_distribuido.head(6)

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13


In [103]:
custo_distribuido['aux'] = custo_distribuido['financeiro'] +custo_distribuido['custo']
custo_distribuido['aux']=custo_distribuido['aux'].apply(int)/100
custo_distribuido[['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
0,2020-04-06,2020-04-08,ENEV3,True,3035.82
1,2020-04-07,2020-04-09,GBIO33,False,-156105.53
2,2020-04-08,2020-04-13,BPAC11,True,7467.83
3,2020-04-08,2020-04-13,GBIO33,False,-99780.17
4,2020-04-08,2020-04-13,PETR4,False,-345304.41
5,2020-04-08,2020-04-13,PETR4,True,-1263.05
6,2020-04-09,2020-04-14,PETR4,False,335210.07
7,2020-04-16,2020-04-20,BPAC11,False,-606508.89
8,2020-04-16,2020-04-20,COGN3,False,-981337.99
9,2020-04-16,2020-04-20,TIET11,False,-479265.07


In [120]:
custo_distribuido[custo_distribuido.daytrade].groupby(['pregao','nome']).sum()

daytrade  qtd  financeiro   turnover         custo  \
pregao     nome                                                         
2020-04-06 ENEV3       True    0      325100   59671100 -21518.000000   
2020-04-08 BPAC11      True    0      770000   76750000 -23216.156106   
           PETR4       True    0     -108000   60516000 -18305.523165   
2020-05-04 OIBR3       True    0     3174600  203174400 -49490.000000   
2020-05-26 BPAC11      True    0    -1475500  201567500 -56987.733437   
           MGLU3       True    0      458100  127840500 -36143.427569   
           PRIO3       True    0     -303600   60836400 -17199.838994   
2020-06-15 BOVA11      True    0        9225       9225     -4.807534   
2020-06-16 BPAC11      True    0       -4700   12842700  -7146.787399   

                        aux  
pregao     nome              
2020-04-06 ENEV3    3035.82  
2020-04-08 BPAC11   7467.83  
           PETR4   -1263.05  
2020-05-04 OIBR3   31251.10  
2020-05-26 BPAC11 -15324.87  
           MGLU3    4219.56  
           PRIO3   -3207.99  
2020-06-15 BOVA11     92.20  
2020-06-16 BPAC11   -118.46

In [182]:
(24.09*98700+custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum())*15/100


49052.765999999996

In [173]:
-custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum()

2050664.56

In [174]:
custo_distribuido[~custo_distribuido.daytrade][custo_distribuido.nome!='SQIA3'].aux.sum()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


351459.52

In [165]:
custo_distribuido[custo_distribuido.daytrade].aux.sum()

26152.139999999996

In [104]:
custo_distribuido[custo_distribuido.nome=='oibr3'.upper()][['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
16,2020-05-04,2020-05-06,OIBR3,True,31251.10
35,2020-06-15,2020-06-17,OIBR3,False,-1019030.78
40,2020-06-18,2020-06-22,OIBR3,False,1096598.12
41,2020-06-23,2020-06-25,OIBR3,False,-514059.86
42,2020-06-26,2020-06-30,OIBR3,False,-232093.07
45,2020-07-24,2020-07-28,OIBR3,False,-50437.04
46,2020-07-31,2020-08-04,OIBR3,False,1184096.06


In [105]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08,3035.82
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09,-156105.53
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13,7467.83
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13,-99780.17
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13,-345304.41
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13,-1263.05
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,-12393.000000,2020-04-14,335210.07
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,-20889.748868,2020-04-20,-606508.89
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,-33799.841068,2020-04-20,-981337.99
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,-16507.139506,2020-04-20,-479265.07


In [107]:
custo_distribuido.groupby('nome').sum()[['qtd']]

,qtd
nome,
BOVA11,0
BPAC11,0
CNTO3,0
COGN3,0
ENEV3,0
GBIO33,0
LWSA3,0
MGLU3,0
OIBR3,0


In [108]:
qtd = aux.groupby(by=['pregao','nome']).sum()[["qtd"]].reset_index().pivot(index='nome',columns='pregao', values='qtd').fillna(0)
qtd

pregao,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-16,2020-04-24,2020-04-29,2020-04-30,2020-05-04,2020-05-06,...,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-23,2020-06-26,2020-07-02,2020-07-08,2020-07-24,2020-07-31
nome,,,,,,,,,,,,,,,,,,,,,
BOVA11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7890.0,0.0,3740.0,4150.0,0.0,0.0,0.0,0.0,0.0,0.0
BPAC11,0.0,0.0,0.0,0.0,15000.0,0.0,0.0,0.0,0.0,0.0,...,-1000.0,-5900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNTO3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COGN3,0.0,0.0,0.0,0.0,180000.0,0.0,-180000.0,40000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENEV3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBIO33,0.0,16400.0,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-26900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LWSA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1100.0,-1100.0,0.0,0.0
MGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OIBR3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1050000.0,0.0,0.0,-1050000.0,443000.0,200000.0,0.0,0.0,30000.0,-673000.0


In [109]:
financeiro = aux.groupby(by=['pregao','nome']).sum()[["financeiro"]].reset_index().pivot(index='nome',columns='pregao', values='financeiro').fillna(0)
financeiro/100

pregao,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-16,2020-04-24,2020-04-29,2020-04-30,2020-05-04,2020-05-06,...,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-23,2020-06-26,2020-07-02,2020-07-08,2020-07-24,2020-07-31
nome,,,,,,,,,,,,,,,,,,,,,
BOVA11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,702155.8,0.0,-340078.2,-385155.0,0.0,0.0,0.0,0.0,0.0,0.0
BPAC11,0.0,0.0,7700.0,0.0,-606300.0,0.0,0.0,0.0,0.0,0.0,...,61360.0,378852.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNTO3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-101120.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COGN3,0.0,0.0,0.0,0.0,-981000.0,0.0,1006200.0,-221600.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENEV3,3251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBIO33,0.0,-155993.0,-99750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,268731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LWSA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-55253.0,49951.0,0.0,0.0
MGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OIBR3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31746.0,0.0,...,-1018500.0,0.0,0.0,1097000.0,-513880.0,-232000.0,0.0,0.0,-50400.0,1184480.0


In [110]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08,3035.82
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09,-156105.53
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13,7467.83
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13,-99780.17
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13,-345304.41
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13,-1263.05
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,-12393.000000,2020-04-14,335210.07
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,-20889.748868,2020-04-20,-606508.89
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,-33799.841068,2020-04-20,-981337.99
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,-16507.139506,2020-04-20,-479265.07


In [621]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


28407.0

In [623]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-2228.6067927113077

In [635]:
#Imposto de renda relativo a daytrade pra ser pago no final do mes 6,referente ao mês 5
(28407.0-2228.6067927113077-12664.86 -12828.95 )*20/100 + taxas_detalhadas.IRRF_daytrade.sum()/100




-268.4633585422617

In [652]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-109637.65

In [650]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-3976.2850533131127